In [4]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import datasets, linear_model, neighbors, svm, naive_bayes
from sklearn.neural_network import MLPRegressor
from sklearn.utils import column_or_1d
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
from sklearn.metrics import classification_report
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import Normalizer
from sklearn.linear_model import LogisticRegression
import re
import sys


def loadGrade():
    csv_path = "grade.csv"
    return pd.read_csv(csv_path, index_col = 0)
# function to read from csv file
def load_data(csv_path):
	# index = not use the first column as the index (row names)
    return pd.read_csv(csv_path, index_col = 0)

# function to return course from argument 2
def read_in():
	course = sys.argv[1];
	# print('course = ',course)
	return course;


def get_rmse(model, X, y):
# model.fit(X, y) = fit model with X and Y
	model.fit(X, y)
# prediction = array with new prediction value from model
	predictions = []
# model.predict = Predict X using the linear model 
	for val in model.predict(X):
		if (val < 0.5):
			predictions.append(0)
		elif (val < 1.5):
			predictions.append(1)
		else:
			predictions.append(2)
# compare predicts array with right value of y1 training data
# lin_mse = error
	lin_mse = mean_squared_error(y, predictions) 
# 	print( " lin_mse ", lin_mse) 
# lin_rmse = square root( căn bậc 2 ) of lin_mse
	lin_rmse = np.sqrt(lin_mse)
# 	print( " lin_rmse ", lin_rmse) 
	return lin_rmse


    
#Load input
# grade from upload
inpgrade = loadGrade()
# names = header of all column
names = list(inpgrade.columns.values)
course = read_in()

# print("course = ", course)

#Rename columns
for name in names:
    newName = re.sub(r"\(.*\)", "", name).strip()
    inpgrade.rename(columns = {name: newName}, inplace = True)
# drop the row of NaN
inpgrade = inpgrade.drop(inpgrade.index[0])
#  drop the fisrt row of point 
inpgrade = inpgrade.drop(inpgrade.index[0])

#  drop the column ID 
if ('ID' in inpgrade.columns.values):
    inpgrade = inpgrade.drop('ID', axis = 1)
#  drop the column Section
if ('Section' in inpgrade.columns.values):
    inpgrade = inpgrade.drop('Section', axis = 1)
# print(" inpgrade section = ")
# print(inpgrade)

# drop all the column of NAN
inpgrade.dropna(axis = 1, how = 'all', inplace = True)
# replace NaN value with 0
inpgrade.fillna(0, inplace = True)
inpgrade['Full name'] = inpgrade.index.values
# choose all column with type of float64 as number

# print("inpgrade = ", inpgrade)
chosen = inpgrade.select_dtypes(include='float64').columns.values

# print("inpgrade.select_dtypes(include='int') = ", inpgrade.select_dtypes(include='int'))

# inpgrade1 = inpgrade.drop(['Student', 'SIS USer ID', 'SIS Login ID'])

inpgrade1 = inpgrade[inpgrade.columns.difference(['Student', 'SIS User ID', 'SIS Login ID', 'Full name'])]


# print('inpgrade1 = ', inpgrade1)

print('colum of inpgrade1 = ', len(inpgrade1.columns) ) 


if ( len(inpgrade1.columns) == 4 ):
	print("hung")    
# 	grade = load_data("../../ml_scripts/data/csce156/MasterTrainingData1.csv") 
	grade = load_data("../../ml_scripts/data/csce156/full-grade-week1.csv")   
    
elif ( len(inpgrade1.columns) == 9 ):   
	grade = load_data("../../ml_scripts/data/csce156/full-grade-week2.csv")
elif ( len(inpgrade1.columns) == 18 ):
	grade = load_data("../../ml_scripts/data/csce156/full-grade-week3.csv")
    
elif ( len(inpgrade1.columns) == 22 ): 
	grade = load_data("../../ml_scripts/data/csce156/full-grade-week4.csv")
#fileO = open("ml_scripts/data/" + course + "/grade.json", "w")
#fileO.write(inpgrade.to_json(orient="records"))
#fileO.close() 


#Load data
#  grade = training data
# grade = load_data("../../ml_scripts/data/csce156/full-grade.csv")
# print('grade = ' , grade.head())
# intersect = ['Homework 1', 'Homework 2', 'Homework 3', ..]
# print("grade= ", grade)
# print('chosen = ', chosen)
#  name of column in chosen in grade from train data
# intersect = [val for val in chosen if val in grade.columns.values]
intersect = [val for val in inpgrade1]
print("intersect = ",intersect)
print("grade = ", grade.tail())
# X = value from full-grade.csv train data 
X = grade[intersect]

poly_degree = 2
Poly_features = PolynomialFeatures(degree=poly_degree, interaction_only=False, include_bias=True)
X_b = Poly_features.fit_transform(X)

print('X = \n')
print( X)


# print('X_b = \n')
# print( X_b)


# Normalize the new feature vector that includes the polynomials
normal_scaler = Normalizer().fit(X_b)
X_normalized = normal_scaler.transform(X_b)


# print('X_normalized = \n')
# print( X_normalized)

# y = true result from train data 
y = grade[["Grade"]].values.ravel()


# predict already in full-grade.csv
y1 = []
for label in y:
	if label == "Good":
		y1.append(0)
	if label == "OK":
		y1.append(1)
	if label == "High-risk":
		y1.append(2)
# print('data = ', y1)




models = []
models.append(LogisticRegression(solver='newton-cg',multi_class='multinomial')) # Logistic Regression
# print(" models 1 = ", models)
models.append(sklearn.naive_bayes.GaussianNB()) # Naive Bayes
# print(" models 2 = ", models)
models.append(sklearn.neighbors.KNeighborsRegressor(n_neighbors=10)) # k Nearest Neighbors
models.append(svm.SVR()) # Support Vector Machine
#models.append(MLPRegressor(hidden_layer_sizes=(60,),activation='logistic',solver='lbfgs',learning_rate='adaptive',max_iter=1000,learning_rate_init=0.01,alpha=0.01)) # Neuron network
models.append(DecisionTreeRegressor()) # Decision Tree
models.append(RandomForestRegressor()) # Random Forest

# print(" models = ", models)
                                                      
clf = LogisticRegression(random_state=0, class_weight='balanced')

print("clf =", clf)
                                                      
                                                      
chosenModels = [None]
minRMSE = float("inf")



for model in models:
# rmse = square root of mean square error (lin_rmse)
	rmse = get_rmse(model, X, y1)
# 	print("model = ", model)
	print("rmse = ", rmse)
	if rmse < minRMSE:
		minRMSE = rmse
		chosenModels[0] = model
# print(" chosenModels = ", chosenModels)
#
lr_pred = []
nb_pred = []
# print(" models[0].predict(X) = ", models[0].predict(X))
# print("X = ", X)
# models[0].predict(X) = linear regression to predict X
for val in models[0].predict(X):
	if (val < 0.5):
# 		print("val < 0.5  = ",val)   
		lr_pred.append(0)
# 		print(" lr_pred < 0.5 = ", lr_pred)       
	elif (val < 1.5):
# 		print("val < 1.5  = ",val)   
		lr_pred.append(1)
# 		print(" lr_pred < 1.5 = ", lr_pred)
	else:
# 		print("val else  = ",val)   
		lr_pred.append(2)
# 		print(" lr_pred else= ", lr_pred)

# print("models[1].predict(X) = ",models[1].predict(X) )

# models[1].predict(X) = naive Bayes to predict X 
for val in models[1].predict(X):
	if (val < 0.5):
		nb_pred.append(0)
	elif (val < 1.5):
		nb_pred.append(1)
	else:
		nb_pred.append(2)
        
# print(" lr_pred = ", lr_pred)
# print(" nb_pred = ", nb_pred)

# lrnb_pred = combination between linear regress and naive bayes
lrnb_pred = []
for i in range(len(lr_pred)):
# lrnb_pred contain the max value of lr_pred and nb_pred
	lrnb_pred.append(max(lr_pred[i], nb_pred[i]))
# print(" lrnb_pred = ", lrnb_pred)
# print('data = ', y1)

# mse = error when compare train data y1 and predict data from lrnb_pred
mse = mean_squared_error(y1, lrnb_pred) 
#  rmse = square root of mse
rmse = np.sqrt(mse)
# print('rmse = ',rmse)
# print('minRMSE = ',minRMSE)
# compare new rmse from lrnb_pred to old minRMSE from 5 models -> take smaller value
if (rmse <= minRMSE):
	chosenModels[0] = models[0]
	chosenModels.append(models[1])
    
# chosenModels[0] = models[0]


# print("models[0].predict(X_normalized) = ",models[0].predict(X_normalized) )


# print("models[0].predict(X) = ",models[0].predict(X) )
    
print("chosenModels = \n" , chosenModels)
print("len(chosenModels)= \n" , len(chosenModels))


#Export models to the file
# chosenModel based on 2 evaluation round: 1st with 5 module and 2nd with lrnb_pred
for i in range(len(chosenModels)):
	joblib.dump(chosenModels[i], '../../ml_scripts/models/csce156/model' + str(i) + '.pkl')
# 	print("str(i) = ", str(i))

# print("model[2] = ", models[2])

# print(models[2].predict([[100,100,100,100,0,100,0,0,0,0,0,0,95,50,0,60,0,18]]))
# print(models[2].predict([[100,100,100,100,99,100,100,100,100,99,100,100,97.5,90,81,85,86,19]]))
# print(models[2].predict([[100,0,100,100,100,100,0,55,100,0,0,0,19,95,44,71,0,21]]))
# print(models[2].predict([[100,100,100,100,99,100,100,100,100,100,100,100,87,93.5,105,98,105,25]]))
print("chosenModels[0] = \n" , chosenModels[0])

chosen_pred = []
for val in chosenModels[0].predict(X):
	if (val < 0.5):
		chosen_pred.append(0)
	elif (val < 1.5):
		chosen_pred.append(1)
	else:
		chosen_pred.append(2)
        
        



res = str(len(chosenModels)) + ";"
#res = ""
for feature in intersect:
	res += feature + ","
# print("res[:-1]  = ", res[:-1])
res = res[:-1] + ";"
# print("res  = ",res[:-1] + ";")
res += inpgrade.to_json(orient="records")
# print("inpgrade.to_json orient=records  =", inpgrade.to_json(orient="records"))
# print("res = ")
# print(res, end='', flush=True)



# print(classification_report(y1,lrnb_pred))

# print(classification_report(y1,lr_pred))

# print(classification_report(y1,nb_pred))






colum of inpgrade1 =  10
intersect =  ['Assignment 1', 'Grade', 'Lab 1', 'Lab 2', 'Lab 3', 'answers', 'contributions', 'notes', 'questions', 'views']
grade =               views  contributions  questions  notes  answers  Lab 1  Lab 2  \
days online                                                                  
9               39              0          0      0        0    100  100.0   
0                0              0          0      0        0    100  100.0   
0                0              0          0      0        0    100  100.0   
0                0              0          0      0        0    100  100.0   
0                0              0          0      0        0    100  100.0   

             Lab 3  Lab 4  Lab 5  Lab 6  Assignment 1  Assignment 2  \
days online                                                           
9            100.0  100.0    100  100.0          90.0          75.0   
0            100.0  100.0    100  100.0          99.0         107.0   
0         

ValueError: could not convert string to float: 'Good'

In [3]:

# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix

# print(" lrnb pred class. report = \n", classification_report(y1,lrnb_pred))
# print(" lrnb pred conf. matrix = \n", confusion_matrix(y1,lrnb_pred))



In [3]:
# print( " nb pred class. report = \n", classification_report(y1,nb_pred))
# print(" nb pred conf. matrix = \n", confusion_matrix(y1,nb_pred))


 nb pred class. report = 
              precision    recall  f1-score   support

          0       0.76      0.86      0.81        37
          1       0.42      0.57      0.48        23
          2       0.76      0.47      0.58        34

avg / total       0.68      0.65      0.65        94

 nb pred conf. matrix = 
 [[32  3  2]
 [ 7 13  3]
 [ 3 15 16]]


In [4]:
# print( " lr pred class. report = \n", classification_report(y1,lr_pred))
# print(" lr pred conf. matrix = \n", confusion_matrix(y1,lr_pred))

 lr pred class. report = 
              precision    recall  f1-score   support

          0       0.77      0.92      0.84        37
          1       0.39      0.30      0.34        23
          2       0.75      0.71      0.73        34

avg / total       0.67      0.69      0.68        94

 lr pred conf. matrix = 
 [[34  3  0]
 [ 8  7  8]
 [ 2  8 24]]
